# Identity Columns

PostgreSQL version 10 introduced a new constraint `GENERATED AS IDENTITY` that allows you to automatically assign a unique number to a column.

The `GENERATED AS IDENTITY` constraint is the SQL standard-conforming variant of the good old `SERIAL` column.

**SYNTAX**
```postgresql
column_name type
GENERATED { ALWAYS | BY DEFAULT }
AS IDENTITY[ ( sequence_option ) ]
```

* The **type** can be `SMALLINT`, `INT`, or `BIGINT`.
* The `GENERATED ALWAYS` instructs PostgreSQL to always generate a value for the identity column.
* If you attempt to insert (or update) values into the `GENERATED ALWAYS AS IDENTITY` column, PostgreSQL will issue an error.
* The `GENERATED BY DEFAULT` instructs PostgreSQL to generate a value for the identity column.
* However, if you supply a value for insert or update, PostgreSQL will use that value to insert into the identity column instead of using the system-generated value.

PostgreSQL allows a table to have more than one identity column. Like the `SERIAL`, the `GENERATED AS IDENTITY` constraint also uses the `SEQUENCE` object internally.

## GENERATED ALWAYS example

```postgresql
CREATE TABLE color (
    color_id INT GENERATED ALWAYS AS IDENTITY,
    color_name VARCHAR NOT NULL
); 


INSERT INTO color(color_name)
VALUES ('Red');                 
-- output: 1, Red


INSERT INTO color (color_id, color_name)
VALUES (2, 'Green');

-- [Err] ERROR:  cannot insert into column "color_id"
-- DETAIL:  Column "color_id" is an identity column defined as GENERATED ALWAYS.
-- HINT:  Use OVERRIDING SYSTEM VALUE to override.

```

To fix the error, you can use the `OVERRIDING SYSTEM VALUE` clause as follows:
```postgresql
INSERT INTO color (color_id, color_name)
OVERRIDING SYSTEM VALUE
VALUES(2, 'Green');
```

Alternatively, you can use GENERATED BY `DEFAULT AS IDENTITY` instead.

## GENERATED BY DEFAULT AS IDENTITY example

```postgresql
DROP TABLE color;

CREATE TABLE color (
    color_id INT GENERATED BY DEFAULT AS IDENTITY,
    color_name VARCHAR NOT NULL
);


INSERT INTO color(color_name)
VALUES ('White');                 
-- output: 1, White

INSERT INTO color (color_id, color_name)
VALUES (2, 'Yellow');
-- output: 2, Yellow

```

## Sequence options example

Because the `GENERATED AS IDENTITY` constraint uses the `SEQUENCE` object, you can specify the sequence options for the system-generated values.

For example, you can specify the starting value and the increment as follows:
```postgresql
DROP TABLE color;

CREATE TABLE color (
    color_id INT GENERATED BY DEFAULT AS IDENTITY
    (START WITH 10 INCREMENT BY 10),
    color_name VARCHAR NOT NULL
);


INSERT INTO color (color_name)
VALUES ('Orange');
-- output: 10, Orange

INSERT INTO color (color_name)
VALUES ('Purple');
-- output: 20, Purple

```

## Adding an identity column to an existing table

You can add identity columns to an existing table by using the following form of the `ALTER TABLE` statement:
```postgresql
ALTER TABLE table_name
ALTER COLUMN column_name
ADD GENERATED { ALWAYS | BY DEFAULT } AS IDENTITY { ( sequence_option ) }
```

**EXAMPLE**
```postgresql
CREATE TABLE shape (
    shape_id INT NOT NULL,
    shape_name VARCHAR NOT NULL
);

ALTER TABLE shape
ALTER COLUMN shape_id ADD GENERATED ALWAYS AS IDENTITY;
```

Note that the shape_id needs to have the `NOT NULL` constraint so that it can be changed to an identity column. 

Otherwise, you’ll get the following error:
```postgresql
ERROR:  column "shape_id" of relation "shape" must be declared NOT NULL before identity can be added
SQL state: 55000
```


## Changing an identity column

You can change the characteristics of an existing identity column by using the following `ALTER TABLE` statement:
```postgresql
ALTER TABLE table_name
ALTER COLUMN column_name
{ SET GENERATED { ALWAYS| BY DEFAULT } |
  SET sequence_option | RESTART [ [ WITH ] restart ] }
```

For example, the following statement changes the **shape_id** column of the **shape** table from GENERATED ALWAYS` to `GENERATED BY DEFAULT`:
```postgresql
ALTER TABLE shape
ALTER COLUMN shape_id SET GENERATED BY DEFAULT;
```



## Removing the GENERATED AS IDENTITY constraint

```postgresql
ALTER TABLE table_name
ALTER COLUMN column_name
DROP IDENTITY [ IF EXISTS ]
```

For example, you can remove the `GENERATED AS IDENTITY` constraint column from the **shape_id** column of the shape table as follows:
```postgresql
ALTER TABLE shape
ALTER COLUMN shape_id
DROP IDENTITY IF EXISTS;
```

# Generated Columns

In PostgreSQL, a **generated column** is a special type of column whose values are automatically calculated based on expressions or values from other columns.

There are two kinds of generated columns:
* Stored: A stored generated column is calculated when it is inserted or updated and occupies storage space.
* Virtual: A virtual generated column is computed when it is read and does not occupy storage space.

A **virtual generated column** is like a **view**, whereas a **stored generated column** is similar to a **materialized view**. 

Unlike a materialized view, PostgreSQL automatically updates data for stored generated columns.

> * **PostgreSQL currently implements only stored generated columns.**
> * **Use generated columns to automate calculations within your table.**

## Defining generated columns

```postgresql
CREATE TABLE table_name(
   ...,
   column_name type GENERATED ALWAYS AS (expression ) STORED | VIRTUAL,
   ...
); 
```
* `STORED` keyword: Indicate that the data of the generated column is physically stored in the table.
* `VIRTUAL` keyword: Indicate that the data of the generated column is computed when queried, not stored physically.

To add a generated column to a table, you can use the ALTER TABLE … ADD COLUMN statement:
```postgresql
ALTER TABLE table_name
ADD COLUMN column_name type GENERATED ALWAYS AS (expression) STORED; 
```

When defining an expression for a generated column, ensure that it meets the following requirements:
* The expression can only use immutable functions and cannot involve **subqueries** or **reference anything beyond the current row**.
* For example, the expression cannot use the `CURRENT_TIMESTAMP` function.
* The expression cannot reference another generated column or a system column, except `tableoid`.

A generated column cannot have a default value or an identity definition. Additionally, it cannot be a part of the partition key.

## Example 1: Concatenating columns

```postgresql
CREATE TABLE contacts(
   id SERIAL PRIMARY KEY,
   first_name VARCHAR(50) NOT NULL,
   last_name VARCHAR(50) NOT NULL,
   full_name VARCHAR(101) GENERATED ALWAYS AS (first_name || ' ' || last_name) STORED,
   email VARCHAR(300) UNIQUE
);

INSERT INTO contacts(first_name, last_name, email)
VALUES
   ('John', 'Doe', 'john.doe@example.com'),
   ('Jane', 'Doe', 'jane.doe@example.com')
RETURNING *;

```

**Output**:
```
id | first_name | last_name | full_name |              email
----+------------+-----------+-----------+---------------------------------
  1 | John       | Doe       | John Doe  | john.doe@example.com
  2 | Jane       | Doe       | Jane Doe  | jane.doe@example.com
(2 rows)
```

## Example 2: Calculating net prices

```postgresql
CREATE TABLE products (
    id SERIAL PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    list_price DECIMAL(10, 2) NOT NULL,
    tax DECIMAL(5, 2) DEFAULT 0,
    discount DECIMAL(5, 2) DEFAULT 0,
    net_price DECIMAL(10, 2) GENERATED ALWAYS AS ((list_price + (list_price * tax / 100)) - (list_price * discount / 100)) STORED
);


INSERT INTO products (name, list_price, tax, discount)
VALUES
    ('A', 100.00, 10.00, 5.00),
    ('B', 50.00, 8.00, 0.00),
    ('C', 120.00, 12.50, 10.00)
RETURNING *;

```

**Output**:
```
id | name | list_price |  tax  | discount | net_price
----+------+------------+-------+----------+-----------
  1 | A    |     100.00 | 10.00 |     5.00 |    105.00
  2 | B    |      50.00 |  8.00 |     0.00 |     54.00
  3 | C    |     120.00 | 12.50 |    10.00 |    123.00
(3 rows)
```